# **Initial**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

drive_root_path = '/content/drive/MyDrive/'

if os.path.exists(drive_root_path):
    print(f"Contents of {drive_root_path}:")
    for item in os.listdir(drive_root_path):
        print(item)
else:
    print(f"The directory '{drive_root_path}' does not exist. Please ensure your Google Drive is mounted correctly.")

In [ ]:
import os

drive_path = '/content/drive/MyDrive/Colab Notebooks/Data Sets/'

if os.path.exists(drive_path):
    print(f"Contents of {drive_path}:")
    for item in os.listdir(drive_path):
        print(item)
else:
    print(f"The directory '{drive_path}' does not exist.")
    print("Please check if 'Colab Notebooks' and 'Data Sets' folders exist in your Google Drive 'MyDrive'.")
    print("You might want to list the contents of '/content/drive/MyDrive/' to debug further.")

In [ ]:
csv_path = "/content/drive/MyDrive/Colab Notebooks/Data Sets/battles.csv"
parquet_path = "/content/drive/MyDrive/Colab Notebooks/Data Sets/battles.parquet"

In [ ]:
import cudf

df = cudf.read_csv(csv_path)
#df.to_parquet(parquet_path)
df.head()

^^^ this will probably take 3 minutes

In [ ]:
df.info()

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'gameMode.id',
                 'winner.clan.tag', 'winner.clan.badgeId',
                 'loser.clan.tag', 'loser.clan.badgeId',
                 'tournamentTag'])

In [ ]:
null_summary = df.isnull().sum().to_pandas()
null_summary = null_summary[null_summary > 0]
print("Null values summary:")
print(null_summary)

In [ ]:
print("\nNumber of duplicate rows:")
df.duplicated().sum()

**Converting Card ID to actual names**

In [ ]:
id_columns = [
    "winner.cards.list","winner.card1.id","winner.card2.id","winner.card3.id","winner.card4.id",
    "winner.card5.id","winner.card6.id","winner.card7.id","winner.card8.id",
    "loser.cards.list","loser.card1.id","loser.card2.id","loser.card3.id","loser.card4.id",
    "loser.card5.id","loser.card6.id","loser.card7.id","loser.card8.id"
]

# ---------------------------
# 2️⃣ Complete ID → Name mapping (all known cards)
# ---------------------------
id_to_name = {
    # Troops
    "Knight": 26000000, "Archers": 26000001, "Goblins": 26000002, "Giant": 26000003,
    "P.E.K.K.A": 26000004, "Minions": 26000005, "Balloon": 26000006, "Witch": 26000007,
    "Barbarians": 26000008, "Golem": 26000009, "Skeletons": 26000010, "Valkyrie": 26000011,
    "Skeleton Army": 26000012, "Bomber": 26000013, "Musketeer": 26000014, "Baby Dragon": 26000015,
    "Prince": 26000016, "Wizard": 26000017, "Mini P.E.K.K.A": 26000018, "Spear Goblins": 26000019,
    "Giant Skeleton": 26000020, "Hog Rider": 26000021, "Minion Horde": 26000022, "Ice Wizard": 26000023,
    "Royal Giant": 26000024, "Guards": 26000025, "Princess": 26000026, "Dark Prince": 26000027,
    "Three Musketeers": 26000028, "Lava Hound": 26000029, "Ice Spirit": 26000030, "Fire Spirits": 26000031,
    "Miner": 26000032, "Sparky": 26000033, "Bowler": 26000034, "Lumberjack": 26000035,
    "Battle Ram": 26000036, "Inferno Dragon": 26000037, "Ice Golem": 26000038, "Mega Minion": 26000039,
    "Dart Goblin": 26000040, "Goblin Gang": 26000041, "Electro Wizard": 26000042,
    "Elite Barbarians": 26000043, "Hunter": 26000044, "Executioner": 26000045, "Bandit": 26000046,
    "Royal Recruits": 26000047, "Night Witch": 26000048, "Bats": 26000049, "Royal Ghost": 26000050,
    "Ram Rider": 26000051, "Zappies": 26000052, "Rascals": 26000053, "Cannon Cart": 26000054,
    "Mega Knight": 26000055, "Skeleton Barrel": 26000056, "Flying Machine": 26000057,
    "Wall Breakers": 26000058, "Royal Hogs": 26000059, "Goblin Giant": 26000060, "Fisherman": 26000061,
    "Magic Archer": 26000062, "Electro Dragon": 26000063, "Firecracker": 26000064,
    "Elixir Golem": 26000067, "Battle Healer": 26000068, "Skeleton Dragons": 26000080,
    "Mother Witch": 26000083, "Electro Spirit": 26000084, "Electro Giant": 26000085,

    # Buildings
    "Cannon": 27000000, "Goblin Hut": 27000001, "Mortar": 27000002, "Inferno Tower": 27000003,
    "Bomb Tower": 27000004, "Barbarian Hut": 27000005, "Tesla": 27000006,
    "Elixir Collector": 27000007, "X-Bow": 27000008, "Tombstone": 27000009,
    "Furnace": 27000010, "Goblin Cage": 27000012,

    # Spells
    "Fireball": 28000000, "Arrows": 28000001, "Rage": 28000002, "Rocket": 28000003,
    "Goblin Barrel": 28000004, "Freeze": 28000005, "Mirror": 28000006, "Lightning": 28000007,
    "Zap": 28000008, "Poison": 28000009, "Graveyard": 28000010, "The Log": 28000011,
    "Tornado": 28000012, "Clone": 28000013, "Earthquake": 28000014, "Barbarian Barrel": 28000015,
    "Heal Spirit": 28000016, "Giant Snowball": 28000017, "Royal Delivery": 28000018
}

# Reverse: ID (as str) → name
id_to_name = {str(v): k for k, v in id_to_name.items()}
id_map = cudf.Series(list(id_to_name.values()), index=list(id_to_name.keys()))

# ---------------------------
# 3️⃣ Preserve original DataFrame for unmapped check
# ---------------------------
df_orig = df[id_columns].copy(deep=True)

# ---------------------------
# 4️⃣ Convert IDs to strings
# ---------------------------
for col in id_columns:
    df[col] = df[col].astype("str")

# ---------------------------
# 5️⃣ Function for list-column replacement
# ---------------------------
def replace_list_column(col):
    cleaned = df[col].str.replace(r"[\[\]]", "", regex=True)
    split_df = cleaned.str.split(",", expand=True)
    for c in split_df.columns:
        split_df[c] = split_df[c].str.strip().map(id_map).fillna(split_df[c])
    combined = split_df[0].fillna("")
    for c in split_df.columns[1:]:
        combined = combined.str.cat(split_df[c].fillna(""), sep=",")
    df[col] = combined.str.replace("^,|,$", "", regex=True).str.replace(",,", ",", regex=True)

# ---------------------------
# 6️⃣ Apply mapping
# ---------------------------
for col in id_columns:
    if "cards.list" in col:
        replace_list_column(col)
    else:
        df[col] = df[col].map(id_map).fillna(df[col])

# ---------------------------
# 7️⃣ Detect unmapped IDs
# ---------------------------
unmapped = cudf.Series([], dtype="object")

for col in id_columns:
    cleaned = df_orig[col].astype("str").str.replace(r"[\[\]]", "", regex=True)
    split_df = cleaned.str.split(",", expand=True)
    for c in split_df.columns:
        tok = split_df[c].str.strip()
        mask_num = tok.str.match(r"^\d+$")
        still_unknown = tok[mask_num & ~tok.isin(id_map.index)]
        unmapped = cudf.concat([unmapped, still_unknown])

unmapped = unmapped.dropna().drop_duplicates().reset_index(drop=True)

#("🧩 IDs that weren't mapped:")
#print(unmapped)

# ---------------------------
# ✅ Quick check output
# ---------------------------
print(df[["loser.card1.id", "winner.cards.list"]].head())

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head().to_pandas()

**Change Arena IDs**

Up to this point (2020), the last arena added was spooky town and then legend arena after that. 12 arenas + 1 legend arena. Thing is, we have 15 diff arenas IDs.

# **Basic Visualizations**

**Trophy Change Distribution**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
plt.figure(figsize=(8,5))
df['winner.trophyChange'].to_pandas().hist(bins=30, alpha=0.6, label='Winner Trophy Change')
df['loser.trophyChange'].to_pandas().hist(bins=30, alpha=0.6, label='Loser Trophy Change')
plt.xlabel('Trophy Change')
plt.ylabel('Frequency')
plt.title('Trophy Change Distribution for Winners and Losers')
plt.legend()
plt.show()


**Crown Earned Distribution**

In [ ]:
plt.figure(figsize=(7,5))
df['winner.crowns'].to_pandas().hist(alpha=0.6, label='Winner Crowns')
df['loser.crowns'].to_pandas().hist(alpha=0.6, label='Loser Crowns')
plt.xlabel('Crowns')
plt.ylabel('Count')
plt.title('Crowns Distribution per Battle')
plt.legend()
plt.show()


**Winner King Tower Health Distribution**

In [ ]:
plt.figure(figsize=(7,5))
df['winner.kingTowerHitPoints'].to_pandas().hist(bins=30, color='teal')
plt.xlabel('King Tower HP')
plt.ylabel('Number of Battles')
plt.title('Distribution of Winner King Tower HP at End of Battle')
plt.show()


**AVG Elixir Cost Comparison**

In [ ]:
plt.figure(figsize=(7,5))
plt.hist(df['winner.elixir.average'].to_pandas(), bins=30, alpha=0.6, label='Winners')
plt.hist(df['loser.elixir.average'].to_pandas(), bins=30, alpha=0.6, label='Losers')
plt.xlabel('Average Elixir Cost')
plt.ylabel('Count')
plt.title('Average Elixir Cost Comparison: Winners vs Losers')
plt.legend()
plt.show()


**Winning Deck Type Composition**

In [ ]:
deck_cols = ['winner.troop.count', 'winner.structure.count', 'winner.spell.count']
deck_means = df[deck_cols].mean().to_pandas()

plt.figure(figsize=(6,5))
deck_means.plot(kind='pie', autopct='%1.1f%%')
plt.title('Average Deck Composition for Winners')
plt.ylabel('')
plt.show()


**Losing Deck Type Composition**

In [ ]:
deck_cols = ['loser.troop.count', 'loser.structure.count', 'loser.spell.count']
deck_means = df[deck_cols].mean().to_pandas()

plt.figure(figsize=(6,5))
deck_means.plot(kind='pie', autopct='%1.1f%%')
plt.title('Average Deck Composition for Losers')
plt.ylabel('')
plt.show()


Losers had slightly more structures than spells. Seems negligible.

**AVG starting Trophies vs Trophy Change**

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(df['winner.startingTrophies'].to_pandas(), df['winner.trophyChange'].to_pandas(), alpha=0.5)
plt.xlabel('Starting Trophies')
plt.ylabel('Trophy Change')
plt.title('Starting Trophies vs Trophy Change (Winners)')
plt.show()


Matchmaking balance. It centers on an average of 30 trophies gain on a win. A plateau exists at ~4000, ~6000, ~8000 trophies. This makes it harder for high-trophy players to move up which makes sense.

**AVG Card Level Difference (Winner vs Loser)**

In [ ]:
df['level_diff'] = df['winner.totalcard.level'] - df['loser.totalcard.level']
plt.figure(figsize=(7,5))
df['level_diff'].to_pandas().hist(bins=50, color='orange')
plt.xlabel('Card Level Difference (Winner - Loser)')
plt.ylabel('Number of Battles')
plt.title('Effect of Card Level Difference on Battle Outcome')
plt.show()


**Most Common Winner Card**

In [ ]:
cards = []
for i in range(1, 9):
    cards.extend(df[f'winner.card{i}.id'].to_pandas().tolist())

pd.Series(cards).value_counts().head(15).plot(kind='bar', figsize=(8,5))
plt.title('Top 15 Most Common Cards in Winning Decks')
plt.xlabel('Card ID')
plt.ylabel('Frequency')
plt.show()


**Correlation Heatmap of Battle Metrics**

In [ ]:
cols = ['winner.startingTrophies', 'winner.totalcard.level', 'winner.elixir.average', 'winner.crowns', 'winner.trophyChange', 'winner.troop.count', 'winner.structure.count', 'winner.spell.count']
corr = df[cols].to_pandas().corr()

plt.figure(figsize=(7,5))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Between Winner Metrics')
plt.show()


**Wins by Arena ID**

In [ ]:
# select winner columns (keep as cudf)
arena_col = 'arena.id'
winner_card_cols = [f'winner.card{i}.id' for i in range(1, 9)]
keep_cols = [arena_col] + winner_card_cols
winner_df = df[keep_cols]

# melt into long form =1 card per row (cudf.melt)
cards_long = winner_df.melt(id_vars=arena_col,
                            value_vars=winner_card_cols,
                            var_name='card_slot',
                            value_name='card_name')

# count card frequency per arena (cudf)
card_freq = cards_long.groupby([arena_col, 'card_name']).size().reset_index(name='count')

# for each arena, pick the card w max count
# sort by arena and descending count, then drop_duplicates(keep='first')
card_freq = card_freq.sort_values([arena_col, 'count'], ascending=[True, False])
top_cards = card_freq.drop_duplicates(subset=[arena_col], keep='first')

# sort arenas numerically or lexicographically for plotting
top_cards = top_cards.sort_values(arena_col).reset_index(drop=True)

# top_cards is a cudf.DataFrame (one row per arena)
# Convert the small result to pandas for plotting
top_pd = top_cards.to_pandas()

x_labels = top_pd[arena_col].astype(str).tolist()
y_counts = top_pd['count'].astype(int).tolist()
card_names = top_pd['card_name'].tolist()

# --- Plot (stacked subplots: bar chart above, 1-row table below) ---
fig, (ax_chart, ax_table) = plt.subplots(
    2, 1, figsize=(14, 6), gridspec_kw={'height_ratios': [3, 0.6]}
)

# Chart
bars = ax_chart.bar(x_labels, y_counts, color='royalblue')
ax_chart.set_title('Most Common Winning Card Frequency per Arena')
ax_chart.set_xlabel('Arena ID')
ax_chart.set_ylabel('Frequency (count of appearances in winning decks)')
ax_chart.grid(axis='y', linestyle='--', alpha=0.4)
ax_chart.tick_params(axis='x', rotation=45)

# labels
for i, v in enumerate(y_counts):
    ax_chart.text(i, v + max(y_counts)*0.01, f"{int(v)}", ha='center', va='bottom', fontsize=8)

# Table
table_data = [card_names]
col_labels = x_labels
row_labels = ['Card Name']

table = ax_table.table(cellText=table_data,
                       rowLabels=row_labels,
                       colLabels=col_labels,
                       cellLoc='center',
                       loc='center')

table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 1.4)

ax_table.axis('off')  # hide axes for table

plt.tight_layout()
plt.show()

Just a guess on the Arena ID mapping, NEED CONFIRMATION

In [ ]:
arena_id_to_name = {
    54000001: "Goblin Stadium",
    54000002: "Bone Pit",
    54000003: "Barbarian Bowl",
    54000004: "P.E.K.K.A's Playhouse",
    54000005: "Spell Valley",
    54000006: "Builder's Workshop",
    54000007: "Royal Arena",
    54000008: "Frozen Peak",
    54000009: "Jungle Arena",
    54000010: "Hog Mountain",
    54000011: "Electro Valley",
    54000024: "Spooky Town", # NEED CONFIRMATION, (24? not 12?)
    54000031: "Legendary Arena",
    54000049: "Treasure of the Old Kings (Season 17)",  # seasonal, NEED CONFIRMATION
    54000050: "Logmas (Season 18)",                     # seasonal, NEED CONFIRMATION
    # next would logically be "Frozen Peak (S19 Seasonal Skin)"
}


# **Visuals on Deck Stats w/ Trophy csv**

**On the Left Hand side, click Files. Click on "upload to session storage". Upload "deck_stats_with_cards_and_trophy.csv".**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from collections import Counter, defaultdict
import matplotlib.gridspec as gridspec

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
data = pd.read_csv('deck_stats_with_cards_and_trophy.csv')
data.head()

**Finding Card Use Frequency**

In [ ]:
# Ensure card columns list
card_cols = [f'c{i}' for i in range(8)]
assert all(col in data.columns for col in card_cols), "Make sure df has c0..c7"

# Create long-form listing of all card occurrences
cards_long = data[card_cols].melt(value_name='card')[['card']]

# Count occurrences
card_counts = cards_long['card'].value_counts().reset_index()
card_counts.columns = ['card_name', 'count']
card_counts = card_counts.sort_values('count', ascending=False).reset_index(drop=True)



# Plot top 10 most frequently used cards
top_n = 10
top_cards = card_counts.head(top_n)
plt.figure(figsize=(10,6))
plt.bar(top_cards['card_name'], top_cards['count'])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Card')
plt.ylabel('Usage Count (in decks)')
plt.title(f'Top {top_n} Most Frequently Used Cards Across Decks')
# annotate counts
for i, v in enumerate(top_cards['count']):
    plt.text(i, v + max(top_cards['count'])*0.01, f"{int(v)}", ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

pd.set_option('display.max_rows', None)
print("\nFull card usage table (descending):")
#print(card_counts)
card_counts.head(10)

Dropped decks with less than 100 games play. This is to prevent games with 100% WR with like 30 games played

In [ ]:
top10 = data[data['total'] >= 100].sort_values('win_rate', ascending=False).head(10).reset_index(drop=True)
top10.head()

In [ ]:
fig = plt.figure(figsize=(12, 8))

# position for the bar chart axis [left, bottom, width, height]
# This places the bar chart in the upper part of the fig
ax_bar = fig.add_axes([0.1, 0.55, 0.8, 0.4])

# table axis
#lower part of the fig, w/ gap from ax_bar
ax_table = fig.add_axes([0.1, 0.45, 0.8, 0.4])

# Bar Chart on ax_bar
bars = ax_bar.bar(top10.index, top10['win_rate'], color='royalblue')
ax_bar.set_xticks(top10.index)
ax_bar.set_xticklabels([f"Deck {i+1}" for i in range(len(top10))], rotation=0)
ax_bar.set_ylabel("Win Rate")
ax_bar.set_title("Top 10 Decks by Win Rate (> 100 Games)")

#win rate values labels
for i, v in enumerate(top10['win_rate']):
    ax_bar.text(i, v + 0.002, f"{v:.2%}", ha='center', va='bottom', fontsize=9)

#Table on ax_table
ax_table.axis('off')

card_cols = [f"c{i}" for i in range(8)]
#rows = Card#, columns = Deck#
table_data = top10[card_cols].T.values.tolist()
row_labels = [f"Card {i+1}" for i in range(8)]
col_labels = [f"Deck {i+1}" for i in range(10)]

# Add table below chart with padding
table = plt.table(cellText=table_data,
                  rowLabels=row_labels,
                  colLabels=col_labels,
                  loc='bottom',
                  cellLoc='center')

# Style and spacing
table.scale(1, 1.4)

plt.show()

In [ ]:
bottom10 = data[data['total'] >= 100].sort_values('win_rate', ascending=True).head(10).reset_index(drop=True)
bottom10.head()

**Invidivual Card's WR Contribution**

For each card, compute average win rate of all decks containing that card.
Insight: Identifies high-impact cards that tend to appear in winning decks.

In [ ]:
# Reshape deck data into long format (1 card per row)
card_cols = [f'c{i}' for i in range(8)]
cards_long = data.melt(value_vars=card_cols, value_name='card', var_name='card_slot', ignore_index=False)

# Merge w/ win rates
cards_long['win_rate'] = data.loc[cards_long.index, 'win_rate'].values

#Group by card and compute mean win rate
card_winrates = cards_long.groupby('card')['win_rate'].mean().reset_index()

card_winrates = card_winrates.sort_values('win_rate', ascending=False).reset_index(drop=True)

print(card_winrates.head(20))  #Top 20 highest win rate cards


**Most Played Decks**

In [ ]:
# Sort by total and take top 10
most_played = data.sort_values('total', ascending=False).head(10).reset_index(drop=True)

# Create figure with 2 stacked subplots (chart + table)
fig, (ax_chart, ax_table) = plt.subplots(
    2, 1, figsize=(12, 9), gridspec_kw={'height_ratios': [2.5, 1]}
)

# --- Chart ---
bars = ax_chart.bar(
    [f'Deck {i+1}' for i in range(len(most_played))],
    most_played['total'],
    color='royalblue'
)
ax_chart.set_title('Top 10 Most Played Decks')
ax_chart.set_xlabel('Deck')
ax_chart.set_ylabel('Total Battles')
ax_chart.tick_params(axis='x', rotation=45)
ax_chart.grid(axis='y', linestyle='--', alpha=0.5)

# label bar
for i, v in enumerate(most_played['total']):
    ax_chart.text(i, v + max(most_played['total']) * 0.01, f'{int(v):,}', ha='center', fontsize=9)

# --- Table ---
card_cols = [f'c{i}' for i in range(8)]
table_data = most_played[card_cols].T.values.tolist()
col_labels = [f'Card {i+1}' for i in range(8)] # had to switch it for transpose
row_labels = [f'Deck {i+1}' for i in range(10)]

table = ax_table.table(
    cellText=table_data,
    rowLabels=col_labels,
    colLabels=row_labels,
    cellLoc='center',
    loc='center'
)

table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 1.4)

# Hide table axes
ax_table.axis('off')

plt.tight_layout()
plt.show()


**Card Pair Co-occurrence Table**

In [ ]:
# Define the card columns
card_cols = [f'c{i}' for i in range(8)]

# Initialize counters
pair_counts = Counter()
pair_win_sum = defaultdict(float)
pair_deck_count = defaultdict(int)

# Iterate through each deck
for idx, row in data.iterrows():
    cards = list(pd.Series(row[card_cols]).dropna().astype(str).unique())
    win_rate = row['win_rate']

    # Generate all unique unordered pairs of cards
    for a, b in itertools.combinations(sorted(cards), 2):
        key = (a, b)
        pair_counts[key] += 1
        pair_win_sum[key] += win_rate
        pair_deck_count[key] += 1

# Build DataFrame for all pairs
pairs_data = []
for (a, b), cnt in pair_counts.items():
    avg_win = pair_win_sum[(a, b)] / pair_deck_count[(a, b)]
    pairs_data.append({
        'card_1': a,
        'card_2': b,
        'count': cnt,
        'avg_win_rate': avg_win
    })

pairs_df = pd.DataFrame(pairs_data)

pairs_by_count = pairs_df.sort_values('count', ascending=False).reset_index(drop=True)
pairs_by_winrate = pairs_df.sort_values('avg_win_rate', ascending=False).reset_index(drop=True)

In [ ]:
print("=== Top 20 Card Pairs by Frequency ===")
pairs_by_count.head(20)


In [ ]:
print("\n=== Top 20 Card Pairs by Average Win Rate ===")
pairs_by_winrate.head(20)

**Archetype WR Comparison (WIP)**

    'Hog Cycle': {'Hog Rider', 'Ice Spirit', < 3.5 elixir},   
    'X-Bow Siege': {'X-Bow', 'Tesla'},     
    'P.E.K.K.A Bridge Spam': {'P.E.K.K.A', 'Bandit', 'Battle Ram'},
    'Miner Poison': {'Miner', 'Poison'},
    'Golem Beatdown': {'Golem', 'P.E.K.K.A'},
    'Balloon Cycle': {'Balloon', 'Freeze', < 3.5 elixir},
    'Royal Giant': {'Royal Giant'},
    'Graveyard': {'Graveyard', 'Poison'},


    need ppl who actually play for input.
    Possibly use the 'wincon' csv from kaggle.

**Working W/ Trophy Bands**

In [ ]:
data['trophy_band'].unique()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ensure the correct order of trophy bands
trophy_order = ['0-3999', '4000-4499', '4500-4999', '5000-5499',
                '5500-5999', '6000-6499', '6500-6999', '7000-7499', '7500-7999']

# --- Step 1: Melt dataset for card frequency analysis ---
cards_long = data.melt(
    id_vars=['trophy_band'],
    value_vars=[f'c{i}' for i in range(8)],
    var_name='card_slot',
    value_name='card'
)

# --- Step 2: Count card frequency per trophy band ---
card_counts = (
    cards_long.groupby(['trophy_band', 'card'])
    .size()
    .reset_index(name='count')
)

# --- Step 3: Find most frequent card per trophy band ---
top_cards = card_counts.loc[card_counts.groupby('trophy_band')['count'].idxmax()]
top_cards = top_cards.set_index('trophy_band').reindex(trophy_order).reset_index()

# --- Step 4: Plot ---
fig, (ax_chart, ax_table) = plt.subplots(
    2, 1, figsize=(12, 8),
    gridspec_kw={'height_ratios': [2.5, 1]}
)

# --- Bar Chart: Frequency of Most Used Card per Trophy Band ---
ax_chart.bar(top_cards['trophy_band'], top_cards['count'], color='royalblue')
ax_chart.set_title('Most Used Card per Trophy Band')
ax_chart.set_xlabel('Trophy Band')
ax_chart.set_ylabel('Frequency')
ax_chart.grid(axis='y', linestyle='--', alpha=0.5)

# Annotate bar values
for i, count in enumerate(top_cards['count']):
    ax_chart.text(i, count + 0.01 * max(top_cards['count']), str(count),
                  ha='center', fontsize=9)

# --- Table: Most Used Card per Trophy Band ---
table_data = [top_cards['card'].tolist()]
col_labels = top_cards['trophy_band'].tolist()
row_labels = ['Most Used Card']

table = ax_table.table(
    cellText=table_data,
    rowLabels=row_labels,
    colLabels=col_labels,
    cellLoc='center',
    loc='center'
)
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1.2, 1.5)
ax_table.axis('off')

plt.tight_layout()
plt.show()


Top 10 Decks by WR (per Trophy Band)

In [ ]:
# if set to total > 20, it will take 3 minutes
# if set to total > 100, fast but last 2 graphs empty

trophy_order = [
    '0-3999', '4000-4499', '4500-4999', '5000-5499',
    '5500-5999', '6000-6499', '6500-6999', '7000-7499', '7500-7999'
]

def deck_color(total):
    if total < 100:
        return 'orange'
    elif total < 500:
        return 'yellow'
    else:
        return 'skyblue'

# Create figure with an outer GridSpec of 9 rows (one per trophy band)
fig = plt.figure(constrained_layout=False, figsize=(14, 56))
outer_gs = gridspec.GridSpec(nrows=9, ncols=1, figure=fig, hspace=0.8)

for i, band in enumerate(trophy_order):
    # Create a sub-gridspec for this band: 2 rows (chart / table)
    sub_gs = outer_gs[i].subgridspec(nrows=2, ncols=1, height_ratios=[3, 1], hspace=0.15)
    ax_bar = fig.add_subplot(sub_gs[0])
    ax_table = fig.add_subplot(sub_gs[1])

    # Filter decks for this band with minimum total
    band_df = data[(data['trophy_band'] == band) & (data['total'] >= 30)].copy()

    if band_df.empty:
        # If no data, hide axes and write a notice
        ax_bar.text(0.5, 0.5, f'No decks with total >= 100 in {band}', ha='center', va='center', fontsize=12)
        ax_bar.set_xticks([])
        ax_bar.set_yticks([])
        ax_table.axis('off')
        ax_bar.set_title(f'{band} — Top 10 Decks by Win Rate', fontsize=11, fontweight='bold')
        continue

    # Select top 10 by win_rate
    top10 = band_df.sort_values('win_rate', ascending=False).head(10).reset_index(drop=True)
    n = len(top10)

    # X labels (Deck 1..n)
    x = np.arange(n)
    x_labels = [f'Deck {j+1}' for j in range(n)]

    # Colors based on total
    colors = [deck_color(t) for t in top10['total']]

    # --- Bar chart ---
    ax_bar.bar(x, top10['win_rate'], color=colors, edgecolor='k', linewidth=0.4)
    ax_bar.set_ylim(0, 1)
    ax_bar.set_xticks(x)
    ax_bar.set_xticklabels(x_labels, rotation=45, ha='right')
    ax_bar.set_ylabel('Win Rate')
    ax_bar.set_title(f'{band} — Top {n} Decks by Win Rate', fontsize=11, fontweight='bold')
    ax_bar.grid(axis='y', linestyle='--', alpha=0.4)


    # Annotate each bar with win_rate as percent
    for xi, rate in zip(x, top10['win_rate']):
        ax_bar.text(xi, rate - 0.02, f"{rate:.2%}", ha='center', va='top', fontsize=9)

    # Optionally add small legend for color meaning (only on the first subplot)
    if i == 0:
        from matplotlib.lines import Line2D
        legend_elements = [
            Line2D([0], [0], color='orange', lw=8, label='total < 500'),
            Line2D([0], [0], color='yellow', lw=8, label='500 ≤ total < 1000'),
            Line2D([0], [0], color='skyblue', lw=8, label='total ≥ 1000'),
        ]
        ax_bar.legend(handles=legend_elements, bbox_to_anchor=(1,1.35), loc='upper right', fontsize=9)

    # --- Table: cards for each deck ---
    # Build table data: rows are Card 1..Card 8, columns are Deck 1..Deck n
    card_rows = []
    for card_idx in range(8):
        col = f'c{card_idx}'
        # ensure string representation
        row_vals = top10[col].astype(str).tolist()
        card_rows.append(row_vals)

    row_labels = [f'Card {k+1}' for k in range(8)]
    col_labels = x_labels

    # Use ax_table.table and hide axes
    table = ax_table.table(
        cellText=card_rows,
        rowLabels=row_labels,
        colLabels=col_labels,
        loc='bottom',
        cellLoc='center'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    # scale the table so contents fit nicely
    table.scale(1.0, 1.2)
    ax_table.axis('off')

# Final layout adjustments
plt.subplots_adjust(top=0.96, bottom=0.05, left=0.05, right=0.98, hspace=0.2)
plt.suptitle('Top Decks by Win Rate per Trophy Band (only decks with total >= 30)', fontsize=16, weight='bold')
plt.show()


**Investigating decks in Higher Trophy Bands**

7000-7499 trophy band

In [ ]:
filtered_7000_7499 = data[data['trophy_band'] == '7000-7499']
sample_7000_7499 = filtered_7000_7499.sample(n=5, random_state=42)
print("Sample of 5 decks from the 7000-7499 trophy band:")
print(sample_7000_7499)

In [ ]:
average_total_7000_7499 = filtered_7000_7499['total'].mean()
print(f"Average 'total' for 7000-7499 trophy band: {average_total_7000_7499}")

In [ ]:
filtered_7000_7499['total'].max()

In [ ]:
display(filtered_7000_7499.head())

7500-7999 trophy band

In [ ]:
filtered_7500_7999 = data[data['trophy_band'] == '7500-7999']
sample_7500_7999 = filtered_7500_7999.sample(n=5, random_state=42)
print("Sample of 5 decks from the 7500-7999 trophy band:")
print(sample_7500_7999)

In [ ]:
average_total_7500_7999 = filtered_7500_7999['total'].mean()
print(f"Average 'total' for 7500-7999 trophy band: {average_total_7500_7999}")

In [ ]:
filtered_7500_7999['total'].max()

In [ ]:
display(filtered_7500_7999.head())